# Clustering for Anomaly Detection


In [1]:
# Data containers and preparation
import data_prep as dp
import numpy as np
import pandas as pd
import pickle as pkl

# Data processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Clustering Methods
from sklearn.cluster import DBSCAN

In [3]:
X, y = dp.load_agg_Xy(path='./data/', strat_cat='attack_cat', sample_size=0.15)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3265: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3265: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# Define the individual steps
ohe_step = ('ohe', OneHotEncoder(sparse=False))
ssc_step = ('std_sclr', StandardScaler())

# Make the step part of a pipeline
ohe_pipe = Pipeline([ohe_step])
ssc_pipe = Pipeline([ssc_step])

# Columns to transform: categorical columns for encoding, numeric feature columns for standardizing
ohe_cols = ['proto', 'state', 'service']
binary_cols = ['is_sm_ips_ports', 'is_ftp_login']
ssc_cols = [col for col in X.columns if col not in ohe_cols+binary_cols]

# Transformer input: tuple w/ contents ('name', SomeTransformer(Parameters), columns)
transformer = [
    ('one_hot_encoding', ohe_pipe, ohe_cols),
    ('standard_scaling', ssc_pipe, ssc_cols)
]
ct = ColumnTransformer(transformers=transformer, remainder='passthrough')

In [5]:
# Save Column Transformer object for later use as these column names and conditions do not change between datasets
with open('ct_ohe_ssc_xyagg.pkl', 'wb') as f:
    pkl.dump(ct,f)

In [ ]:
db = DBSCAN(eps=0.35, min_samples=10).fit(ct.fit_transform(X))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:
pd.Series(db.labels_).unique().size

In [11]:
y.value_counts()

normal            332815
generic            32322
exploits            6679
fuzzers             3637
dos                 2453
reconnaissance      2098
analysis             402
backdoors            349
shellcode            227
worms                 26
Name: attack_cat, dtype: int64